In [ ]:
from dotenv import load_dotenv
import os

from langchain_tavily import TavilySearch
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode, create_react_agent


In [ ]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:
def get_gemini_model(temp=0.6):
    return ChatGoogleGenerativeAI(
        model="gemini-2.5-pro",
        temperature=temp,
        google_api_key=GOOGLE_API_KEY
    )


In [ ]:
@tool
def arithmetic_tool(expression: str) -> str:
    """Evaluate a basic arithmetic expression and return the result."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {e}"


In [ ]:
@tool
def summarize_text(text: str) -> str:
    """Summarize the provided text into a concise statement."""
    model = get_gemini_model(0.3)
    summary = model.invoke(f"Summarize the following text in 2-3 sentences:\n\n{text}")
    return getattr(summary, "content", str(summary))


In [ ]:
@tool
def suggest_medications(disease: str) -> str:
    """Suggest possible medications for the given disease (for educational purposes only)."""
    model = get_gemini_model(0.3)
    meds = model.invoke(f"Suggest common medications for {disease}. "
                        "This is for informational purposes only and not medical advice.")
    return getattr(meds, "content", str(meds))


In [ ]:
@tool
def decision_helper(situation: str) -> str:
    """Suggest a possible decision for the given situation."""
    model = get_gemini_model(0.6)
    decision = model.invoke(f"Given the following situation, suggest a reasonable decision: {situation}")
    return getattr(decision, "content", str(decision))


In [ ]:
search = TavilySearch(max_results=2)
tools = [search, arithmetic_tool, summarize_text, suggest_medications, decision_helper]


memory = MemorySaver()
main_agent = create_react_agent(get_gemini_model(0.7), tools, checkpointer=memory)


workflow = StateGraph(dict)

In [ ]:
def agent_node(state):
    print(">>> Running Main Agent...")
    response = main_agent.invoke(
        {"messages": state["messages"]},
        config={"configurable": {"thread_id": state["thread_id"]}}
    )
    state["agent_response"] = response
    return state

workflow.add_node("agent", agent_node)

workflow.add_edge("agent", END)

workflow.set_entry_point("agent")

app = workflow.compile()


In [3]:
result = app.invoke({
    "messages": [("user", "I am in a situation where I need to decide whether to invest in a new business venture or save my money. What should I do?")],
    "thread_id": "session_1"
})
print("\nFinal Result:")

print(result["agent_response"])


>>> Running Main Agent...

Final Result:
{'messages': [HumanMessage(content='I am in a situation where I need to decide whether to invest in a new business venture or save my money. What should I do?', additional_kwargs={}, response_metadata={}, id='08580a05-aa84-4a2c-a158-f86a5a76e4f7'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'decision_helper', 'arguments': '{"situation": "Deciding whether to invest in a new business venture or save money."}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-pro', 'safety_ratings': []}, id='run--c2ac0bbb-c482-44eb-8212-4d7fc26326e7-0', tool_calls=[{'name': 'decision_helper', 'args': {'situation': 'Deciding whether to invest in a new business venture or save money.'}, 'id': '7fd0b9e8-5322-466c-917c-aa2385329b5c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1439, 'output_tokens': 177, 'total_tokens': 1616, 'input_token_details': {'